In [1]:
import argparse
d = {"pop": 100}
# Make dict into args
args = argparse.Namespace(**d)
args.pop

100

In [2]:
save_path = "/home/paul/debug"

import jax.numpy as jnp
import json

# Load config
print(save_path + "_config.json")
with open(save_path + "_config.json", "r") as f:
    config = json.load(f)
    # Lists to tuples
    for k, v in config.items():
        if isinstance(v, list):
            config[k] = tuple(v)
    config = argparse.Namespace(**config)
    print(config)

/home/paul/debug_config.json
Namespace(env_name='walker2d_uni', episode_length=100, evals=100, seed=42, policy_hidden_layer_sizes=(16, 16), critic_hidden_layer_sizes=(64, 64), num_init_cvt_samples=50000, num_centroids=1024, min_bd=0.0, max_bd=1.0, es='cmaes', pop=10, es_sigma=0.01, sample_mirror=True, sample_rank_norm=True, adam_optimizer=True, learning_rate=0.01, l2_coefficient=0.02, nses_emitter=False, novelty_nearest_neighbors=10, rl=True, testrl=True, carlies=False, elastic_pull=0.001, actor_injection=True, nb_injections=1, critic_training=1000, pg_training=1000, output='debug', plot=False, wandb='', tag='', jobid='', log_period=1, debug=True, num_gens=10, algo='CMAES-RL-TestRL-AI', config='ES 10 - σ 0.01 - α 0.01 - ε 0.001')


In [3]:
from qdax.core.rl_es_parts.es_setup import setup_es

Device count: 1 [GpuDevice(id=0, process_index=0)]


In [4]:
EM = setup_es(config)

Imported modules
Doing actor injection x 1


/home/paul/Dev/QDax/qdax/core/rl_es_parts/es_utils.py:253: UserWarning: This type of repertoire does not store the extra scores computed by the scoring function
  repertoire = ESRepertoire.init(


layer_shapes [(16,), (17, 16), (16,), (16, 16), (6,), (16, 6)]
sizes [ 16 272  16 256   6  96]
Initialized ES


In [5]:
es = EM.es,
env = EM.env,
policy_network = EM.policy_network,
emitter = EM.emitter,
emitter_state = EM.emitter_state,
repertoire = EM.repertoire,
random_key = EM.random_key,
wandb_run  = EM. wandb_run,

In [6]:
offspring = jnp.load(save_path + "offspring.npy")
actor = jnp.load(save_path + "_actor.npy")
critic = jnp.load(save_path + "_critic.npy")

In [7]:
offspring.shape

(662,)

In [8]:
import jax 
from qdax import environments
from jax.flatten_util import ravel_pytree

from qdax.core.neuroevolution.networks.networks import MLP

policy_layer_sizes = config.policy_hidden_layer_sizes + (env.action_size,)
policy_network = MLP(
    layer_sizes=policy_layer_sizes,
    kernel_init=jax.nn.initializers.lecun_uniform(),
    final_activation=jnp.tanh,
)


random_key = jax.random.PRNGKey(0)
keys = jax.random.split(random_key, num=1)

env = environments.create(config.env_name, episode_length=config.episode_length)

fake_batch = jnp.zeros(shape=(1, env.observation_size))
init_variables = jax.vmap(policy_network.init)(keys, fake_batch)

_, reconstruction_fn = ravel_pytree(init_variables)

AttributeError: 'tuple' object has no attribute 'action_size'

In [ ]:
reconstruction_fn(actor)